In [2]:
import tweepy
import numpy as np
import pandas as pd
import re
from google.cloud import language_v1
import geoplot as gplt
import geopandas as gpd
import geoplot.crs as gcrs
import time

In [2]:
with open("api_key.txt") as consumer_keys :
    consumer_key = consumer_keys.readline().strip()
    consumer_secret = consumer_keys.readline().strip()
    
with open("access_tokens.txt") as access_tokens :
    access_token = access_tokens.readline().strip()
    access_token_secret = access_tokens.readline().strip()

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth,
                 wait_on_rate_limit=True,
                 wait_on_rate_limit_notify = True,
                 retry_count = 5,
                 retry_delay = 5)

In [3]:
# get the latitude and longitude for all US cities 
# credit - https://simplemaps.com/data/us-cities
lat_long = pd.read_csv('uscities.csv', usecols=['lat','lng'])

In [7]:
list_of_tweets = []
latitude = []
longitude = []
row = 0
while True :
    try:
        for index in range(row, len(lat_long)) :
            lat = str(lat_long.iloc[index, 0])
            long = str(lat_long.iloc[index, 1])
            tweet_list = api.search(q='-is:retweet -RT',
                                    geocode= lat + ',' + long + ',' + '25mi',
                                    lang='en', result_type='recent', count=100)
            latitude.extend([lat_long.iloc[index, 0] for i in range(len(tweet_list))])
            longitude.extend([lat_long.iloc[index, 1] for i in range(len(tweet_list))])
            list_of_tweets.append(tweet_list)
            row = index + 1
    except tweepy.error.TweepError:
        time.sleep(120)
        continue
    break

In [20]:
flatten = lambda list_of_lists : [tweet for sublist in list_of_lists for tweet in sublist]

In [ ]:
tweets = flatten(list_of_tweets)

In [31]:
text = []
for tweet in range(len(tweet_list)) :
    temp = re.sub("(\@\S*) | (https\://[a-z0-9A-Z\.\-_/]*)", "", tweet_list[tweet].text)
    temp = re.sub("#", " ", temp)
    temp = re.sub("(\t)|(\n)", " ", temp)
    text.append(re.sub("[^a-z0-9A-Z!\-\.\ ]", "", temp))

In [32]:
twitter_data = pd.DataFrame({'lat':latitude, 'long':longitude, 'text':text})
twitter_data.to_csv('twitter_data.csv', index=False)
twitter_data = twitter_data.dropna()
twitter_data = twitter_data.reset_index(drop=True)

In [2]:
client = language_v1.LanguageServiceClient()

In [16]:
sentiment_magnitude = []
sentiment_score = []
for index, row in twitter_data.iterrows() :
    time.sleep(0.1) # to not exceed API limits
    document = language_v1.Document(content=row['text'], language='en', type_=language_v1.Document.Type.PLAIN_TEXT)
    sentiment = client.analyze_sentiment(request={'document': document}).document_sentiment
    sentiment_score.append(sentiment.score)
    sentiment_magnitude.append(sentiment.magnitude)

In [26]:
twitter_data['score'] = sentiment_score
twitter_data['magnitude'] = sentiment_magnitude

In [31]:
groups = twitter_data.groupby(['lat', 'long'])
city_sent_avg = groups.apply(lambda x: np.average(x.score, weights=x.magnitude))

map_data = pd.DataFrame({'intensity':city_sent_avg})
map_data.reset_index(inplace=True)

In [ ]:
geo_df = gpd.GeoDataFrame(map_data, geometry=gpd.points_from_xy(map_data.long, map_data.lat))
gplt.pointplot(geo_df, projection=gcrs.AlbersEqualArea(), hue='score', legend=True)

In [ ]:
us_map = gpd.read_file(gplt.datasets.get_path('contiguous_usa'))
ax = geoplot.voronoi(geo_df,  
                     hue='score',  
                     clip=us_map,  
                     projection=gcrs.AlbersEqualArea(),  
                     cmap='RdYlGn',  
                     k=None,  
                     legend=True, 
                     edgecolor='white',  
                     linewidth=0.01  
                    )
geoplot.polyplot(us_map,  
                 ax=ax,  
                 extent=USA.total_bounds, 
                 edgecolor='white', 
                 linewidth=0.01,  
                 zorder=1  
                 )